In [2]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

# Define the KMV model class
class KMVModel:
    def __init__(self, market_cap, shares, stock_price, assets, current_liabilities, non_current_liabilities, risk_free_rate, sigma, T=1):
        self.market_cap = market_cap
        self.shares = shares
        self.stock_price = stock_price
        self.assets = assets
        self.current_liabilities = current_liabilities
        self.non_current_liabilities = non_current_liabilities
        self.risk_free_rate = risk_free_rate
        self.sigma = sigma
        self.T = T
        self.DPT = self.current_liabilities + 0.5 * self.non_current_liabilities
        self.iterations = 0

    def calculate_d1_d2(self, V):
        """Calculate d1 and d2 for given asset value V."""
        d1 = (np.log(V / self.DPT) + (self.risk_free_rate + self.sigma**2 / 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return d1, d2

    def fair_value(self, V):
        """Calculate the fair value of equity for given asset value V."""
        d1, d2 = self.calculate_d1_d2(V)
        FV = V * norm.cdf(d1) - self.DPT * np.exp(-self.risk_free_rate * self.T) * norm.cdf(d2)
        return FV
    
    def mispricing(self, V):
        """Calculate the mispricing given asset value V."""
        FV = self.fair_value(V)
        mispricing = (FV - self.market_cap)**2
        return mispricing

    def optimize_assets(self):
        """Optimize the asset value to minimize mispricing."""
        result = minimize(lambda V: self.mispricing(V), x0=self.market_cap, bounds=[(0, None)])
        if result.success:
            optimized_V = result.x[0]
        else:
            raise ValueError("Optimization failed.")
        return optimized_V

    def distance_to_default(self, V):
        """Calculate the distance to default given asset value V."""
        d1, d2 = self.calculate_d1_d2(V)
        DD = d2
        return DD

    def probability_of_default(self, V):
        """Calculate the probability of default given asset value V."""
        DD = self.distance_to_default(V)
        PD = norm.cdf(-DD)
        return PD

# Placeholder data from the transcription, replace with real values
market_cap = 14.89234  # Market capitalization in billions of dollars
shares = 303.6  # Shares outstanding in millions
stock_price = 49.052502  # Stock price in dollars
assets = 3.4993  # Total assets in billions of dollars
current_liabilities = 1.3547  # Current liabilities in billions of dollars
non_current_liabilities = 0.5421  # Non-current liabilities in billions of dollars
risk_free_rate = 0.0405  # Risk-free rate in decimals
sigma = 0.6546  # Asset volatility in decimals
T = 1  # Time horizon in years

# Initialize the KMV model
kmv_model = KMVModel(market_cap, shares, stock_price, assets, current_liabilities, non_current_liabilities, risk_free_rate, sigma, T)

# Start the optimization to find the asset value that minimizes mispricing
optimized_assets = kmv_model.optimize_assets()

# Calculate the distance to default and probability of default
distance_to_default = kmv_model.distance_to_default(optimized_assets)
probability_of_default = kmv_model.probability_of_default(optimized_assets)

(optimized_assets, distance_to_default, probability_of_default)

(16.453437778391965, 3.270416454474676, 0.0005369462912450246)

In [7]:
import numpy as np
from scipy.stats import norm

# Constants given in the formula
STD = 500_000  # Short-term debt
LTD = 900_000  # Long-term debt
A = 1_000_000    # Current asset value
sigma_A = 0.15  # Asset volatility
r = 0.0645      # Risk-free rate
T = 1         # Time horizon in years

# Calculating the default point (DPT)
DPT = STD + 0.5 * LTD

# Calculating d1 and d2 using the given formulas
d1 = (np.log(A / DPT) + (r + sigma_A ** 2 / 2) * T) / (sigma_A * np.sqrt(T))
d2 = d1 - sigma_A * np.sqrt(T)        

# Calculating the firm value (FV) at time T
FV = A * norm.cdf(d1) - DPT * np.exp(-r * T) * norm.cdf(d2)
    
# Calculating the Expected Default Frequency (EDF)
EDF = norm.cdf(-d2)

FV, d1, d2, EDF

print(f"Fair Value (at time T): {FV:.2f}")
print(f"Expected Default Frequency (EDF): {EDF:.2%}")

Fair Value (at time T): 127185.52
Expected Default Frequency (EDF): 24.29%
